In [1]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/Alzheimer-MRI-Classification-Project/research'

In [2]:
os.chdir('../')

In [13]:
%pwd

'/home/towet/Desktop/OpenProjects/Alzheimer-MRI-Classification-Project'

In [14]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: Path 
    local_data_file: Path 
    unzip_dir: Path 
    

In [15]:
from AlzheimerClassifier.constants import *
from AlzheimerClassifier.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from AlzheimerClassifier import logger

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
    

In [19]:
try:
    config = ConfigurationManager()
    ingestion_config = config.get_data_ingestion_config()
    obj = DataIngestion(config=ingestion_config)
    obj.download_file()
    obj.extract_zip_file()
except Exception as e:
    raise e

[2024-02-08 20:12:31,888: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-08 20:12:31,917: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-08 20:12:31,918: INFO: common: created directory at: artifacts]
[2024-02-08 20:12:31,920: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-08 20:12:31,921: INFO: 1008487374: Downloading data from https://drive.google.com/file/d/1xBrgJTMB9VxYFw2Aof28INrIyVHoGknd/view?usp=sharing into file artifacts/data_ingestion/alzheimer.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1xBrgJTMB9VxYFw2Aof28INrIyVHoGknd
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1xBrgJTMB9VxYFw2Aof28INrIyVHoGknd&confirm=t&uuid=6c11395e-3402-4fbb-9ce2-73fc298bd217
To: /home/towet/Desktop/OpenProjects/Alzheimer-MRI-Classification-Project/artifacts/data_ingestion/alzheimer.zip
100%|██████████| 398M/398M [00:42<00:00, 9.35MB/s] 

[2024-02-08 20:13:16,604: INFO: 1008487374: Downloaded data from https://drive.google.com/file/d/1xBrgJTMB9VxYFw2Aof28INrIyVHoGknd/view?usp=sharing into file artifacts/data_ingestion/alzheimer.zip]
